 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8889/notebooks/14-full-res-model-v2.ipynb#Load-libraries" data-toc-modified-id="Load-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load libraries</a></span></li><li><span><a href="http://localhost:8889/notebooks/14-full-res-model-v2.ipynb#Define-loss-functions" data-toc-modified-id="Define-loss-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define loss functions</a></span></li><li><span><a href="http://localhost:8889/notebooks/14-full-res-model-v2.ipynb#Define-models" data-toc-modified-id="Define-models-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define models</a></span></li><li><span><a href="http://localhost:8889/notebooks/14-full-res-model-v2.ipynb#Save-predictions-of-512-model" data-toc-modified-id="Save-predictions-of-512-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Save predictions of 512 model</a></span></li><li><span><a href="http://localhost:8889/notebooks/14-full-res-model-v2.ipynb#Full-res-model" data-toc-modified-id="Full-res-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Full res model</a></span></li><li><span><a href="http://localhost:8889/notebooks/14-full-res-model-v2.ipynb#Training" data-toc-modified-id="Training-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="http://localhost:8889/notebooks/14-full-res-model-v2.ipynb#Full-res-model-2" data-toc-modified-id="Full-res-model-2-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Full res model 2</a></span></li></ul></div>

# Load libraries

In [1]:
import cv2
import numpy as np
import pandas as pd

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, BatchNormalization
from keras.optimizers import RMSprop
from keras.losses import binary_crossentropy
import keras.backend as K

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
import random
import gzip
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

# Define loss functions

In [3]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

# Define models

In [4]:
def unet_down_one_block(inputs, num_filters):
    x = Conv2D(num_filters, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [5]:
def unet_max_pool(inputs):
    x = MaxPooling2D((2, 2), strides=(2, 2))(inputs)
    return x

In [6]:
def unet_up_one_block(up_input, down_input, num_filters):
    x = UpSampling2D((2,2))(up_input)
    x = concatenate([down_input, x], axis=3)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [7]:
def get_unet(input_shape = (256, 256, 3),
             num_classes = 1,
             initial_filters = 32,
             central_filters = 1024):
    
    num_filters = initial_filters
    
    out_list    = [Input(shape=input_shape)]
    down_interim_list = []
    
    while num_filters <= central_filters/2:
        x = unet_down_one_block(out_list[-1], num_filters)
        down_interim_list.append(x)
        num_filters = num_filters * 2
        y = unet_max_pool(x)
        out_list.append(y)
    
    x = unet_down_one_block(out_list[-1], num_filters)
    out_list.append(x)
    num_filters = int(num_filters / 2)
    
    while num_filters >= initial_filters:
        x = unet_up_one_block(out_list[-1], down_interim_list.pop(), num_filters)
        out_list.append(x)
        num_filters = int(num_filters / 2)
    
    classify = Conv2D(num_classes, (1,1), activation = 'sigmoid')(out_list[-1])
    
    model = Model(inputs=out_list[0], outputs=classify)
    
    model.compile(optimizer=RMSprop(lr=0.0001),
                  loss=bce_dice_loss,
                  metrics=[dice_coeff])
    
    return model

# Save predictions of 512 model

In [8]:
model_512 = get_unet(input_shape=(512,512,3), initial_filters=16)

In [9]:
model_512.load_weights('./weights/best_weights_512_2.hdf5')

In [10]:
input_size=512

In [11]:
from os import listdir
ids_list = [x.split('.')[0] for x in listdir('data/train')]

In [12]:
all_imgs=[]
for id in ids_list:
    img = cv2.imread('data/train/{}.jpg'.format(id))
    img = cv2.resize(img, (input_size, input_size))
    all_imgs.append(img)

In [13]:
preds = model_512.predict(np.array(all_imgs, np.float32)/255, batch_size=16)
preds = np.squeeze(preds, axis=3)

In [14]:
preds.shape

(5088, 512, 512)

In [15]:
for i in range(len(ids_list)):
    this_id = ids_list[i]
    this_pred = np.copy(preds[i,:,:])
    f = gzip.open('./data/preds512/' + this_id+ '.pklz','wb')
    pickle.dump(this_pred, f)
    f.close()

# Full res model

In [8]:
model_fullres =  get_unet(input_shape=(1280,1920,4), initial_filters=8, central_filters=64)

In [9]:
model_fullres.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1280, 1920, 4) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 1280, 1920, 8) 296         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 1280, 1920, 8) 32          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 1280, 1920, 8) 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [10]:
from keras.utils import plot_model  
plot_model(model_fullres, to_file='model-fullres-v2.png', show_shapes=True, show_layer_names=True)  
from IPython.display import FileLink
FileLink('model-fullres-v2.png')

/home/sainath/extvol/kaggle-carvana-image-masking-challenge/model-fullres-v2.png

# Training

In [10]:
threshold  = 0.5

In [11]:
df_train = pd.read_csv('data/train_masks.csv')

In [12]:
ids_train = df_train['img'].map(lambda s: s.split('.')[0])

In [13]:
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)

In [14]:
def randomHueSaturationValue(image, hue_shift_limit=(-180, 180),
                             sat_shift_limit=(-255, 255),
                             val_shift_limit=(-255, 255), u=0.5):
    if np.random.random() < u:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(image)
        hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
        h = cv2.add(h, hue_shift)
        sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
        s = cv2.add(s, sat_shift)
        val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
        v = cv2.add(v, val_shift)
        image = cv2.merge((h, s, v))
        image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)

    return image

In [15]:
def randomShiftScaleRotate(image, mask,
                           shift_limit=(-0.0625, 0.0625),
                           scale_limit=(-0.1, 0.1),
                           rotate_limit=(-45, 45), aspect_limit=(0, 0),
                           borderMode=cv2.BORDER_CONSTANT, u=0.5):
    if np.random.random() < u:
        height, width, channel = image.shape

        angle = np.random.uniform(rotate_limit[0], rotate_limit[1])  # degree
        scale = np.random.uniform(1 + scale_limit[0], 1 + scale_limit[1])
        aspect = np.random.uniform(1 + aspect_limit[0], 1 + aspect_limit[1])
        sx = scale * aspect / (aspect ** 0.5)
        sy = scale / (aspect ** 0.5)
        dx = round(np.random.uniform(shift_limit[0], shift_limit[1]) * width)
        dy = round(np.random.uniform(shift_limit[0], shift_limit[1]) * height)

        cc = np.math.cos(angle / 180 * np.math.pi) * sx
        ss = np.math.sin(angle / 180 * np.math.pi) * sy
        rotate_matrix = np.array([[cc, -ss], [ss, cc]])

        box0 = np.array([[0, 0], [width, 0], [width, height], [0, height], ])
        box1 = box0 - np.array([width / 2, height / 2])
        box1 = np.dot(box1, rotate_matrix.T) + np.array([width / 2 + dx, height / 2 + dy])

        box0 = box0.astype(np.float32)
        box1 = box1.astype(np.float32)
        mat = cv2.getPerspectiveTransform(box0, box1)
        image = cv2.warpPerspective(image, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                    borderValue=(
                                        0, 0,
                                        0,))
        mask = cv2.warpPerspective(mask, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                   borderValue=(
                                       0, 0,
                                       0,))

    return image, mask

In [16]:
def randomHorizontalFlip(image, mask, u=0.5):
    if np.random.random() < u:
        image = cv2.flip(image, 1)
        mask = cv2.flip(mask, 1)

    return image, mask

In [17]:
def m512_stuff(ids_list):
    final_512_preds=[]
    for this_id in ids_list:
        
        f = gzip.open('./data/preds512/' + this_id + '.pklz','rb')
        mask = pickle.load(f)
        f.close()
        
        resized_mask = cv2.resize(mask, (1920, 1280))
        resized_mask = np.expand_dims(resized_mask, axis=3)*255
        final_512_preds.append(resized_mask)
    
    return final_512_preds

In [18]:
def train_generator(train_batch_size):
    while True:
        this_ids_train_split = random.sample(list(ids_train_split), len(list(ids_train_split)))
        for start in range(0, len(ids_train_split), train_batch_size):
            x_batch = []
            y_batch = []
            
            start=0
            end = min(start + train_batch_size, len(ids_train_split))
            ids_train_batch = this_ids_train_split[start:end]
            
            all_imgs = []
            for id in ids_train_batch:
                img = cv2.imread('data/train/{}.jpg'.format(id))
                all_imgs.append(img)
            
            final_512_preds = m512_stuff(ids_train_batch)
            
            for i in range(len(all_imgs)):
                img = cv2.resize(all_imgs[i], (1920,1280))
                img = randomHueSaturationValue(img,
                                               hue_shift_limit=(-50, 50),
                                               sat_shift_limit=(-5, 5),
                                               val_shift_limit=(-15, 15))
                img = np.concatenate([img, final_512_preds[i]], axis=2)
                
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(ids_train_batch[i]), cv2.IMREAD_GRAYSCALE)
                mask = cv2.resize(mask, (1920,1280))
                
                img, mask = randomShiftScaleRotate(img, mask,
                                                   shift_limit=(-0.0625, 0.0625),
                                                   scale_limit=(-0.1, 0.1),
                                                   rotate_limit=(-0, 0))
                img, mask = randomHorizontalFlip(img, mask)
                mask = np.expand_dims(mask, axis=2)
                x_batch.append(img)
                y_batch.append(mask)
            
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [19]:
def valid_generator(val_batch_size):
    while True:
        for start in range(0, len(ids_valid_split), val_batch_size):
            
            x_batch = []
            y_batch = []
            
            end = min(start + val_batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            
            all_imgs = []
            for id in ids_valid_batch.values:
                img = cv2.imread('data/train/{}.jpg'.format(id))
                all_imgs.append(img)
            
            final_512_preds = m512_stuff(ids_valid_batch.values)
            
            for i in range(len(all_imgs)):
                img = cv2.resize(all_imgs[i], (1920,1280))
                img = np.concatenate([img, final_512_preds[i]], axis=2)
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(ids_valid_batch.values[i]), cv2.IMREAD_GRAYSCALE)
                mask = cv2.resize(mask, (1920,1280))
                mask = np.expand_dims(mask, axis=2)
                x_batch.append(img)
                y_batch.append(mask)
            
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [20]:
max_epochs = 50

In [21]:
train_batch_size = 3
val_batch_size   = 8

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/best_weights_fullresv2.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

model_fullres.fit_generator(generator=train_generator(train_batch_size),
                            steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                            epochs=max_epochs,
                            verbose=2,
                            callbacks=callbacks,
                            validation_data=valid_generator(val_batch_size),
                            validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/50
4466s - loss: 0.7622 - dice_coeff: 0.5814 - val_loss: 0.6156 - val_dice_coeff: 0.6557
Epoch 2/50
4459s - loss: 0.4551 - dice_coeff: 0.7366 - val_loss: 0.3884 - val_dice_coeff: 0.7606
Epoch 3/50
4459s - loss: 0.2361 - dice_coeff: 0.8573 - val_loss: 0.2059 - val_dice_coeff: 0.8662
Epoch 4/50
4460s - loss: 0.2182 - dice_coeff: 0.8479 - val_loss: 0.1380 - val_dice_coeff: 0.9107
Epoch 5/50
4460s - loss: 0.0832 - dice_coeff: 0.9425 - val_loss: 0.0594 - val_dice_coeff: 0.9633
Epoch 6/50
4460s - loss: 0.0290 - dice_coeff: 0.9808 - val_loss: 0.0294 - val_dice_coeff: 0.9848
Epoch 7/50
4459s - loss: 0.0129 - dice_coeff: 0.9913 - val_loss: 0.0186 - val_dice_coeff: 0.9920
Epoch 8/50
4458s - loss: 0.0083 - dice_coeff: 0.9943 - val_loss: 0.0174 - val_dice_coeff: 0.9937
Epoch 9/50
4460s - loss: 0.0049 - dice_coeff: 0.9973 - val_loss: 0.0141 - val_dice_coeff: 0.9953
Epoch 10/50
4461s - loss: 0.0045 - dice_coeff: 0.9975 - val_loss: 0.0148 - val_dice_coeff: 0.9953
Epoch 11/50
4461s - loss: 0.0

KeyboardInterrupt: 

# Full res model 2

In [8]:
model_fullres =  get_unet(input_shape=(1280,1920,4), initial_filters=8, central_filters=1024)

In [9]:
model_fullres.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1280, 1920, 4) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 1280, 1920, 8) 296         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 1280, 1920, 8) 32          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 1280, 1920, 8) 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/best_weights_fullresv3.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

model_fullres.fit_generator(generator=train_generator(train_batch_size),
                            steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                            epochs=max_epochs,
                            verbose=2,
                            callbacks=callbacks,
                            validation_data=valid_generator(val_batch_size),
                            validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/50
5437s - loss: 0.6051 - dice_coeff: 0.6865 - val_loss: 0.5555 - val_dice_coeff: 0.6788
Epoch 2/50
5413s - loss: 0.5022 - dice_coeff: 0.6988 - val_loss: 0.3965 - val_dice_coeff: 0.7570
Epoch 3/50
5411s - loss: 0.1670 - dice_coeff: 0.8868 - val_loss: 0.1190 - val_dice_coeff: 0.9239
Epoch 5/50
5414s - loss: 0.0846 - dice_coeff: 0.9406 - val_loss: 0.0596 - val_dice_coeff: 0.9635
Epoch 6/50
5414s - loss: 0.0291 - dice_coeff: 0.9803 - val_loss: 0.0271 - val_dice_coeff: 0.9856
Epoch 7/50
5415s - loss: 0.0101 - dice_coeff: 0.9936 - val_loss: 0.0176 - val_dice_coeff: 0.9929
Epoch 8/50
5416s - loss: 0.0048 - dice_coeff: 0.9971 - val_loss: 0.0148 - val_dice_coeff: 0.9950
Epoch 9/50
5415s - loss: 0.0036 - dice_coeff: 0.9978 - val_loss: 0.0150 - val_dice_coeff: 0.9954
Epoch 10/50
5416s - loss: 0.0030 - dice_coeff: 0.9986 - val_loss: 0.0139 - val_dice_coeff: 0.9958
Epoch 11/50
5418s - loss: 0.0026 - dice_coeff: 0.9986 - val_loss: 0.0145 - val_dice_coeff: 0.9958
Epoch 12/50
5417s - loss: 0.

KeyboardInterrupt: 

In [26]:
model_fullres.load_weights('./weights/best_weights_fullresv3.hdf5')

In [27]:
model_fullres.compile(optimizer=RMSprop(lr=0.00001),
                  loss=bce_dice_loss,
                  metrics=[dice_coeff])

In [28]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=4,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=3,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/best_weights_fullresv3.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

model_fullres.fit_generator(generator=train_generator(train_batch_size),
                            steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                            epochs=max_epochs,
                            verbose=2,
                            callbacks=callbacks,
                            validation_data=valid_generator(val_batch_size),
                            validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/50
5415s - loss: 0.0028 - dice_coeff: 0.9985 - val_loss: 0.0153 - val_dice_coeff: 0.9956
Epoch 2/50


ResourceExhaustedError: OOM when allocating tensor with shape[3,24,1280,1920]
	 [[Node: training_1/RMSprop/gradients/conv2d_35/convolution_grad/Conv2DBackpropInput = Conv2DBackpropInput[T=DT_FLOAT, _class=["loc:@conv2d_35/convolution"], data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](training_1/RMSprop/gradients/conv2d_35/convolution_grad/Shape, conv2d_35/kernel/read, training_1/RMSprop/gradients/AddN_18)]]

Caused by op 'training_1/RMSprop/gradients/conv2d_35/convolution_grad/Conv2DBackpropInput', defined at:
  File "/home/sainath/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/sainath/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/sainath/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/sainath/anaconda3/lib/python3.